In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy

In [2]:
from utility import get_spotify_client, get_spotify_oauth

In [4]:
%%capture
sp = get_spotify_oauth()

In [ ]:
with open('similarity')